# **Example**: Splitting Structured Documents by Header Levels with `HeaderSplitter`

Large HTML or Markdown documents often contain multiple sections delineated by headers (`<h1>`, `<h2>`, `#`, `##`, etc.). Chunking these documents by their headers makes them easier to process, search, or send to an LLM. **SplitterMR’s `HeaderSplitter` (or `TagSplitter`) allows you to define *semantic* header levels and split documents accordingly—without manual regex or brittle parsing.**

This Splitter class implements two different Langchain text splitters. See documentation below:

- [HTML Header Text Splitter](https://python.langchain.com/api_reference/text_splitters/html/langchain_text_splitters.html.HTMLHeaderTextSplitter.html)
- [Markdown Header Text Splitter](https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/)

## Splitting HTML Files

### Step 1: Read an HTML File

We will use the [**`VanillaReader`**](https://andreshere00.github.io/Splitter_MR/api_reference/reader/#vanillareader) to load a sample HTML file:

In [1]:
from splitter_mr.reader import VanillaReader

file = "https://raw.githubusercontent.com/andreshere00/Splitter_MR/refs/heads/main/data/webpage_example.html"
reader = VanillaReader()  # you can use the argument html_to_markdown=True
reader_output = reader.read(file)

# Print metadata and content
print(reader_output.model_dump_json(indent=4))

{
    "text": "<!DOCTYPE html>\n  <html lang='en'>\n  <head>\n    <meta charset='UTF-8'>\n    <meta name='viewport' content='width=device-width, initial-scale=1.0'>\n    <title>Fancy Example HTML Page</title>\n  </head>\n  <body>\n    <h1>Main Title</h1>\n    <p>This is an introductory paragraph with some basic content.</p>\n    \n    <h2>Section 1: Introduction</h2>\n    <p>This section introduces the topic. Below is a list:</p>\n    <ul>\n      <li>First item</li>\n      <li>Second item</li>\n      <li>Third item with <strong>bold text</strong> and <a href='#'>a link</a></li>\n    </ul>\n    \n    <h3>Subsection 1.1: Details</h3>\n    <p>This subsection provides additional details. Here's a table:</p>\n    <table border='1'>\n      <thead>\n        <tr>\n          <th>Header 1</th>\n          <th>Header 2</th>\n          <th>Header 3</th>\n        </tr>\n      </thead>\n      <tbody>\n        <tr>\n          <td>Row 1, Cell 1</td>\n          <td>Row 1, Cell 2</td>\n          <td>Row 


The `text` attribute contains the raw HTML, including headers, paragraphs, lists, tables, images, and more:

In [2]:
print(reader_output.text)

<!DOCTYPE html>
  <html lang='en'>
  <head>
    <meta charset='UTF-8'>
    <meta name='viewport' content='width=device-width, initial-scale=1.0'>
    <title>Fancy Example HTML Page</title>
  </head>
  <body>
    <h1>Main Title</h1>
    <p>This is an introductory paragraph with some basic content.</p>
    
    <h2>Section 1: Introduction</h2>
    <p>This section introduces the topic. Below is a list:</p>
    <ul>
      <li>First item</li>
      <li>Second item</li>
      <li>Third item with <strong>bold text</strong> and <a href='#'>a link</a></li>
    </ul>
    
    <h3>Subsection 1.1: Details</h3>
    <p>This subsection provides additional details. Here's a table:</p>
    <table border='1'>
      <thead>
        <tr>
          <th>Header 1</th>
          <th>Header 2</th>
          <th>Header 3</th>
        </tr>
      </thead>
      <tbody>
        <tr>
          <td>Row 1, Cell 1</td>
          <td>Row 1, Cell 2</td>
          <td>Row 1, Cell 3</td>
        </tr>
        <tr>
      


### Step 2: Split the HTML File by Header Levels

We create a `HeaderSplitter` and specify which semantic headers to split on (e.g., `"Header 1"`, `"Header 2"`, `"Header 3"`). There are up to 6 levels of headers available:

In [3]:
from splitter_mr.splitter import HeaderSplitter

splitter = HeaderSplitter(headers_to_split_on=["Header 1", "Header 2", "Header 3"])
splitter_output = splitter.split(reader_output)

for idx, chunk in enumerate(splitter_output.chunks):
    print("=" * 40 + f" Chunk {idx + 1} " + "=" * 40 + "\n" + chunk + "\n")

======================================== Chunk 1 ========================================
html
Fancy Example HTML Page

======================================== Chunk 2 ========================================
# Main Title  
This is an introductory paragraph with some basic content.

======================================== Chunk 3 ========================================
## Section 1: Introduction  
This section introduces the topic. Below is a list:  
- First item
- Second item
- Third item with **bold text** and [a link](#)

======================================== Chunk 4 ========================================
### Subsection 1.1: Details  
This subsection provides additional details. Here's a table:  
| Header 1 | Header 2 | Header 3 |
| --- | --- | --- |
| Row 1, Cell 1 | Row 1, Cell 2 | Row 1, Cell 3 |
| Row 2, Cell 1 | Row 2, Cell 2 | Row 2, Cell 3 |

======================================== Chunk 5 ========================================
## Section 2: Media Content  
This se


Each chunk corresponds to a logical section or sub-section in the HTML, grouped by headers and their associated content.


---

## Splitting Markdown File

The exact same interface works for Markdown files. Just change the path:

In [4]:
print("Markdown file example")

file = "https://raw.githubusercontent.com/andreshere00/Splitter_MR/refs/heads/main/data/markdown_example.md"
reader = VanillaReader()
reader_output = reader.read(file)

print(reader_output.model_dump_json(indent=4))

Markdown file example
{
    "text": "---\n__Advertisement :)__\n\n- __[pica](https://nodeca.github.io/pica/demo/)__ - high quality and fast image\n  resize in browser.\n- __[babelfish](https://github.com/nodeca/babelfish/)__ - developer friendly\n  i18n with plurals support and easy syntax.\n\nYou will like those projects!\n\n---\n\n# h1 Heading 8-)\n## h2 Heading\n### h3 Heading\n#### h4 Heading\n##### h5 Heading\n###### h6 Heading\n\n\n## Horizontal Rules\n\n___\n\n---\n\n***\n\n\n## Typographic replacements\n\nEnable typographer option to see result.\n\n(c) (C) (r) (R) (tm) (TM) (p) (P) +-\n\ntest.. test... test..... test?..... test!....\n\n!!!!!! ???? ,,  -- ---\n\n\"Smartypants, double quotes\" and 'single quotes'\n\n\n## Emphasis\n\n**This is bold text**\n\n__This is bold text__\n\n*This is italic text*\n\n_This is italic text_\n\n~~Strikethrough~~\n\n\n## Blockquotes\n\n\n> Blockquotes can also be nested...\n>> ...by using additional greater-than signs right next to each other..

In [5]:
print(reader_output.text)

---
__Advertisement :)__

- __[pica](https://nodeca.github.io/pica/demo/)__ - high quality and fast image
  resize in browser.
- __[babelfish](https://github.com/nodeca/babelfish/)__ - developer friendly
  i18n with plurals support and easy syntax.

You will like those projects!

---

# h1 Heading 8-)
## h2 Heading
### h3 Heading
#### h4 Heading
##### h5 Heading
###### h6 Heading


## Horizontal Rules

___

---

***


## Typographic replacements

Enable typographer option to see result.

(c) (C) (r) (R) (tm) (TM) (p) (P) +-

test.. test... test..... test?..... test!....

!!!!!! ???? ,,  -- ---

"Smartypants, double quotes" and 'single quotes'


## Emphasis

**This is bold text**

__This is bold text__

*This is italic text*

_This is italic text_

~~Strikethrough~~


## Blockquotes


> Blockquotes can also be nested...
>> ...by using additional greater-than signs right next to each other...
> > > ...or with spaces between arrows.


## Lists

Unordered

+ Create a list by starting a lin

The original markdown file is:

```md
---
__Advertisement :)__

- __[pica](https://nodeca.github.io/pica/demo/)__ - high quality and fast image
  resize in browser.
- __[babelfish](https://github.com/nodeca/babelfish/)__ - developer friendly
  i18n with plurals support and easy syntax.

You will like those projects!

---

# h1 Heading 8-)
## h2 Heading
### h3 Heading
#### h4 Heading
##### h5 Heading
###### h6 Heading


## Horizontal Rules

___

---

***


## Typographic replacements

Enable typographer option to see result.

(c) (C) (r) (R) (tm) (TM) (p) (P) +-

test.. test... test..... test?..... test!....

!!!!!! ???? ,,  -- ---

"Smartypants, double quotes" and 'single quotes'


## Emphasis

**This is bold text**

__This is bold text__

*This is italic text*

_This is italic text_

~~Strikethrough~~


## Blockquotes


> Blockquotes can also be nested...
>> ...by using additional greater-than signs right next to each other...
> > > ...or with spaces between arrows.

...
```


To split this text by the level 2 headers (`##`), we can use the following instructions:

In [6]:
splitter = HeaderSplitter(headers_to_split_on=["Header 2"])
splitter_output = splitter.split(reader_output)

for idx, chunk in enumerate(splitter_output.chunks):
    print("=" * 40 + f" Chunk {idx + 1} " + "=" * 40 + "\n" + chunk + "\n")

======================================== Chunk 1 ========================================
---
__Advertisement :)__  
- __[pica](https://nodeca.github.io/pica/demo/)__ - high quality and fast image
resize in browser.
- __[babelfish](https://github.com/nodeca/babelfish/)__ - developer friendly
i18n with plurals support and easy syntax.  
You will like those projects!  
---  
# h1 Heading 8-)  
## h2 Heading
### h3 Heading
#### h4 Heading
##### h5 Heading
###### h6 Heading

======================================== Chunk 2 ========================================
## Horizontal Rules  
___  
---  
***

======================================== Chunk 3 ========================================
## Typographic replacements  
Enable typographer option to see result.  
(c) (C) (r) (R) (tm) (TM) (p) (P) +-  
test.. test... test..... test?..... test!....  
!!!!!! ???? ,,  -- ---  
"Smartypants, double quotes" and 'single quotes'

======================================== Chunk 4 =====================


**And that's it!** 

Note that `## h2 Heading` is not picked as an actual header since there is no blankline between `##` and the end of the title. Test with other Headers as your choice!

## Complete Script

```python
from splitter_mr.reader import VanillaReader
from splitter_mr.splitter import HeaderSplitter

# Step 1: Read the HTML file
print("HTML file example")
file = "https://raw.githubusercontent.com/andreshere00/Splitter_MR/refs/heads/main/data/webpage_example.html"
reader = VanillaReader() # you can use the argument html_to_markdown=True to transform directly to markdown
reader_output = reader.read(file)
print(reader_output.model_dump_json(indent=4))
print(reader_output.text)

splitter = HeaderSplitter(headers_to_split_on=["Header 1", "Header 2", "Header 3"])
splitter_output = splitter.split(reader_output)
for idx, chunk in enumerate(splitter_output.chunks):
    print("="*40 + f" Chunk {idx + 1} " + "="*40 + "\n" + chunk + "\n")

# Step 2: Read the Markdown file
print("Markdown file example")
file = "https://raw.githubusercontent.com/andreshere00/Splitter_MR/refs/heads/main/data/markdown_example.md"
reader = VanillaReader()
reader_output = reader.read(file)
print(reader_output.model_dump_json(indent=4))
print(reader_output.text)

splitter = HeaderSplitter(headers_to_split_on=["Header 2"])
splitter_output = splitter.split(reader_output)
for idx, chunk in enumerate(splitter_output.chunks):
    print("="*40 + f" Chunk {idx + 1} " + "="*40 + "\n" + chunk + "\n")
```